In [1]:
from pprint import pprint
import pandas as pd

In [2]:
import praw

user_agent = "Scraper 1.0 by /u/danieljagun"
reddit = praw.Reddit(
    client_id="K-AfR5cf3qvXEXM_MTn7IQ",
    client_secret="JdAlNMb6dPkWCd4bX4PdXCwIpbA5gQ",
    user_agent=user_agent
)

# hot new rising top
headlines = set()
for submission in reddit.subreddit('CryptoCurrency').hot(limit=None):
    print(submission.title)
    print(submission.id)
    print(submission.author)
    print(submission.created_utc)
    print(submission.score)
    print(submission.upvote_ratio)
    print(submission.url)
    break
    headlines.add(submission.title)
print(len(headlines))

In [3]:
# hot new rising top
headlines = set()
for submission in reddit.subreddit('CryptoCurrency').hot(limit=None):
    headlines.add(submission.title)
print(len(headlines))

333


In [4]:
df = pd.DataFrame(headlines)
df.head()

,0
0,"Daily Crypto Discussion - October 25, 2023 (GM..."
1,World Bank Tokenizes $100 Million Bond on Euro...
2,What happened to the Ethereum triple halvening?
3,Could BlackRock manipulate the market and cras...
4,InvoiceMate Partners with DFINITY to Migrate t...


In [5]:
df.to_csv('titles.csv', header=False, encoding='utf-8', index=False)

In [6]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/daniel/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [7]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as Senti

senia = Senti()
results = []

for line in headlines:
    pol_score = senia.polarity_scores(line)
    pol_score['headline'] = line
    results.append(pol_score)

pprint(results[:3], width=100)

[{'compound': 0.0,
  'headline': 'Daily Crypto Discussion - October 25, 2023 (GMT+0)',
  'neg': 0.0,
  'neu': 1.0,
  'pos': 0.0},
 {'compound': 0.3182,
  'headline': 'World Bank Tokenizes $100 Million Bond on Euroclear’s New Blockchain Solution',
  'neg': 0.0,
  'neu': 0.813,
  'pos': 0.187},
 {'compound': 0.0,
  'headline': 'What happened to the Ethereum triple halvening?',
  'neg': 0.0,
  'neu': 1.0,
  'pos': 0.0}]


In [8]:
df = pd.DataFrame.from_records(results)
df.head()

,neg,neu,pos,compound,headline
0,0.000,1.000,0.000,0.0000,"Daily Crypto Discussion - October 25, 2023 (GM..."
1,0.000,0.813,0.187,0.3182,World Bank Tokenizes $100 Million Bond on Euro...
2,0.000,1.000,0.000,0.0000,What happened to the Ethereum triple halvening?
3,0.348,0.652,0.000,-0.5859,Could BlackRock manipulate the market and cras...
4,0.000,1.000,0.000,0.0000,InvoiceMate Partners with DFINITY to Migrate t...


In [9]:
df['label'] = 0
df.loc[df['compound'] > 0.2, 'label'] = 1
df.loc[df['compound'] < -0.2, 'label'] = -1
df.head()

,neg,neu,pos,compound,headline,label
0,0.000,1.000,0.000,0.0000,"Daily Crypto Discussion - October 25, 2023 (GM...",0
1,0.000,0.813,0.187,0.3182,World Bank Tokenizes $100 Million Bond on Euro...,1
2,0.000,1.000,0.000,0.0000,What happened to the Ethereum triple halvening?,0
3,0.348,0.652,0.000,-0.5859,Could BlackRock manipulate the market and cras...,-1
4,0.000,1.000,0.000,0.0000,InvoiceMate Partners with DFINITY to Migrate t...,0


In [10]:
df2 = df[['headline', 'label']]

In [11]:
df2.to_csv('reddit_title_label.csv', encoding='utf-8', index=False)

In [12]:
df.label.value_counts()

label
 0    175
 1     94
-1     64
Name: count, dtype: int64

In [13]:
print("Positive Titles:\n")
pprint(list(df[df['label'] == 1].headline)[:5], width=200)

print("Negative Titles:\n")
pprint(list(df[df['label'] == -1].headline)[:5], width=200)

Positive Titles:

['World Bank Tokenizes $100 Million Bond on Euroclear’s New Blockchain Solution',
 "Roblox Debunks 'Inaccurate' XRP Support Claims, Says Crypto Payments Not Allowed - Decrypt",
 'Bitcoin dominance hits highest point in over two years',
 'A letter to crypto media: Slow down, be thoughtful, and get the facts right',
 'Spot Bitcoin ETF Excitement Hits Main Street, Google Search Indicates']
Negative Titles:

['Could BlackRock manipulate the market and crash USDT to buy low?',
 "This isn't the first time fake news in crypto has burned investors, Elon Musk also duped the crypto world and everyone quietly forgot about it.",
 'The secret life of Jimmy Zhong, who stole – and lost – more than $3b',
 'Fantom Foundation Wallets Drained; $657K Stolen',
 'US Chamber of Digital Commerce Backs Binance, Urges SEC to Drop Lawsuit']
